In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv    # official GAT implementation in PyG
from torch_geometric.datasets import Planetoid 
import torch_geometric.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt 

name_data = 'Cora'
dataset = Planetoid(root='data',name = name_data)
dataset.transform = T.NormalizeFeatures()

print(f"Number of Classes in {name_data}:", dataset.num_classes)
print(f"Number of Node Features in {name_data}:", dataset.num_node_features)

Number of Classes in Cora: 7
Number of Node Features in Cora: 1433


Processing...
Done!


In [3]:
from models.gmodel import GAT, VGAT


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT(dataset.num_features,10,dataset.num_classes).to(device)
data = dataset[0].to(device)
print(data)

optimizer = torch.optim.Adam(model.parameters(),lr=0.005,weight_decay=5e-4)



Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
def train(model,optimizer,data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return model,loss.item()
    
def test(model,optimizer,data):
    model.eval()
    logits,accs = model(data),[]
    for _,mask in data('train_mask','val_mask','test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [5]:
def train_loop(model,optimizer,data,epochs):

    best_val_acc = test_acc = 0

    for epoch in range(1,epochs):
        model,loss = train(model,optimizer,data)
        _,val_acc,tmp_test_acc = test(model,optimizer,data)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        log = 'Epoch: {}, TrainLoss:{}, Val: {:.4f}, Test: {:.4f}'
        
        if epoch%10==0:
            print(log.format(epoch,loss,best_val_acc,test_acc))

In [6]:
train_loop(model,optimizer,data,1000)

Epoch: 10, TrainLoss:1.940522313117981, Val: 0.0720, Test: 0.0640
Epoch: 20, TrainLoss:1.938442587852478, Val: 0.0720, Test: 0.0640
Epoch: 30, TrainLoss:1.9369033575057983, Val: 0.1840, Test: 0.0640
Epoch: 40, TrainLoss:1.9336224794387817, Val: 0.2700, Test: 0.1440
Epoch: 50, TrainLoss:1.9073584079742432, Val: 0.4440, Test: 0.3190
Epoch: 60, TrainLoss:1.8982778787612915, Val: 0.4440, Test: 0.3190
Epoch: 70, TrainLoss:1.8735800981521606, Val: 0.4440, Test: 0.3190
Epoch: 80, TrainLoss:1.8544108867645264, Val: 0.4440, Test: 0.3190
Epoch: 90, TrainLoss:1.8561052083969116, Val: 0.4440, Test: 0.3190
Epoch: 100, TrainLoss:1.8558294773101807, Val: 0.4440, Test: 0.3190
Epoch: 110, TrainLoss:1.7937495708465576, Val: 0.4440, Test: 0.3190
Epoch: 120, TrainLoss:1.8027818202972412, Val: 0.4440, Test: 0.3190
Epoch: 130, TrainLoss:1.7930790185928345, Val: 0.4440, Test: 0.3190
Epoch: 140, TrainLoss:1.6798571348190308, Val: 0.4440, Test: 0.3190
Epoch: 150, TrainLoss:1.7902414798736572, Val: 0.4440, Test

KeyboardInterrupt: 